# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233288366951                   -0.50    8.0
  2   -7.249768009179       -1.78       -1.40    1.0
  3   -7.251310445032       -2.81       -1.96    4.0
  4   -7.251321738864       -4.95       -2.41    5.0
  5   -7.251331870825       -4.99       -2.62    4.0
  6   -7.251338681964       -5.17       -3.58    4.0
  7   -7.251338646563   +   -7.45       -3.57    3.0
  8   -7.251338794902       -6.83       -4.33    2.0
  9   -7.251338798657       -8.43       -5.40    2.0
 10   -7.251338798698      -10.38       -5.62    4.0
 11   -7.251338798704      -11.23       -6.05    2.0
 12   -7.251338798704      -12.23       -6.46    2.0
 13   -7.251338798705      -12.84       -7.19    3.0
 14   -7.251338798705      -14.27       -7.49    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.02910623808805482
[ Info: Arnoldi iteration step 2: normres = 0.43825587408787353
[ Info: Arnoldi iteration step 3: normres = 0.34650488457361434
[ Info: Arnoldi iteration step 4: normres = 0.9742699252743563
[ Info: Arnoldi iteration step 5: normres = 0.5782043739460957
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.60e-01, 7.87e-02, 3.89e-01, 2.11e-01, 5.26e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2941533309143249
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (9.71e-02, 1.03e-01, 2.31e-01, 9.95e-02, 1.78e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09136366950058385
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.42e-03, 1.09e-02, 1.79e-02, 5.14e-02, 6.86e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11823754439894017
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.26e